In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\avina\onedrive\desktop\ml_env\venv\lib\site-packages (0.0)


You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(42)

In [2]:
import pandas as pd
import numpy as np


In [3]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df = df.loc[:,~df.columns.str.contains('err')]
df

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,FALSE POSITIVE,0,0,0,1,8.589871,132.016100,0.765,4.80600,87.7,...,929,176.40,8.4,1,5638,4.296,1.088,298.74921,46.973351,14.478
6987,FALSE POSITIVE,0,1,1,0,0.527699,131.705093,1.252,3.22210,1579.2,...,2088,4500.53,453.3,1,5638,4.529,0.903,297.18875,47.093819,14.082
6988,CANDIDATE,0,0,0,0,1.739849,133.001270,0.043,3.11400,48.5,...,1608,1585.81,10.6,1,6119,4.444,1.031,286.50937,47.163219,14.757
6989,FALSE POSITIVE,0,0,1,0,0.681402,132.181750,0.147,0.86500,103.6,...,2218,5713.41,12.3,1,6173,4.447,1.041,294.16489,47.176281,15.385


In [6]:
# df.rename(columns = {"koi_disposition": "Disposition", 'koi_fpflag_nt': "NotTransit", "koi_fpflag_ss":"Stellar Eclipse", "koi_fpflag_co":"Centroid Offset", "koi_fpflag_ec":"Ephemeris Match"})

# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
selected_features = df[["koi_disposition", "koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec", "koi_prad"]]
selected_features

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_prad
0,CONFIRMED,0,0,0,0,2.83
1,FALSE POSITIVE,0,1,0,0,14.60
2,FALSE POSITIVE,0,1,0,0,33.46
3,CONFIRMED,0,0,0,0,2.75
4,CONFIRMED,0,0,0,0,2.77
...,...,...,...,...,...,...
6986,FALSE POSITIVE,0,0,0,1,1.11
6987,FALSE POSITIVE,0,1,1,0,29.35
6988,CANDIDATE,0,0,0,0,0.72
6989,FALSE POSITIVE,0,0,1,0,1.07


# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
X = selected_features.drop(["koi_disposition"], axis=1)
y = selected_features["koi_disposition"]
print(X.shape, y.shape)

(6991, 5) (6991,)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [11]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_prad
3563,0,0,0,0,3.89
4099,0,0,0,0,2.10
5460,0,0,0,0,14.59
1091,0,0,0,0,2.28
5999,0,0,0,0,2.27


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [27]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [28]:
# # Step 1: Label-encode data set
# label_encoder = LabelEncoder()
# label_encoder.fit(y_train)
# encoded_y_train = label_encoder.transform(y_train)
# encoded_y_test = label_encoder.transform(y_test)

In [29]:
# # Step 2: Convert encoded labels to one-hot-encoding
# y_train_categorical = to_categorical(encoded_y_train)
# y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [31]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [32]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 200)               1200      
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 603       
Total params: 42,003
Trainable params: 42,003
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/60
5243/5243 - 0s - loss: 0.5322 - accuracy: 0.7011
Epoch 2/60
5243/5243 - 0s - loss: 0.4066 - accuracy: 0.7406
Epoch 3/60
5243/5243 - 0s - loss: 0.4064 - accuracy: 0.7391
Epoch 4/60
5243/5243 - 0s - loss: 0.4053 - accuracy: 0.7402
Epoch 5/60
5243/5243 - 0s - loss: 0.4072 - accuracy: 0.7440
Epoch 6/60
5243/5243 - 0s - loss: 0.4066 - accuracy: 0.7423
Epoch 7/60
5243/5243 - 0s - loss: 0.4053 - accuracy: 0.7467
Epoch 8/60
5243/5243 - 0s - loss: 0.4055 - accuracy: 0.7438
Epoch 9/60
5243/5243 - 0s - loss: 0.4054 - accuracy: 0.7397
Epoch 10/60
5243/5243 - 0s - loss: 0.4049 - accuracy: 0.7459
Epoch 11/60
5243/5243 - 0s - loss: 0.4054 - accuracy: 0.7448
Epoch 12/60
5243/5243 - 0s - loss: 0.4047 - accuracy: 0.7482
Epoch 13/60
5243/5243 - 0s - loss: 0.4047 - accuracy: 0.7414
Epoch 14/60
5243/5243 - 0s - loss: 0.4049 - accuracy: 0.7421
Epoch 15/60
5243/5243 - 0s - loss: 0.4041 - accuracy: 0.7461
Epoch 16/60
5243/5243 - 0s - loss: 0.4047 - accuracy: 0.7473
Epoch 17/60

In [35]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 0s - loss: 0.3465 - accuracy: 0.7637
Normal Neural Network - Loss: 0.37623430103131783, Accuracy: 0.763729989528656


In [36]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [37]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED'
 'FALSE POSITIVE' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED'
 'CONFIRMED' 'FALSE POSITIVE']
Actual Labels: ['CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Train the model with GridSearch


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'avinash_dhawan.sav'
joblib.dump(your_model, filename)